In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Libraries
import os
import pickle

from src.models import train_model, test_model, predict_model, build_model

In [3]:
os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
# os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

In [4]:
EXPERIMENT = 'mh02'
MODEL = 'modelA'
EXP_DIR = f'./models/{EXPERIMENT}/'
# Training
PATH_TO_DATA = f'data/processed/synthetic_data/{MODEL}/'
init_occupancy = 'expected'
# Models
model_template = f'./models/templates/{MODEL}.pl'
model = f'./models/{EXPERIMENT}/{MODEL}.pl'

In [9]:
pk_dict = train_model.get_pk(PATH_TO_DATA)
build_model.add_pk(model_template, model, pk_dict)

In [5]:
exp1 = {'evidence': ['activity', 'fold_change', 'occupancy'], 'facts': []}
exp2 = {'evidence': ['fold_change', 'occupancy'], 'facts': ['activity']}
exp3 = {'evidence': ['activity', 'occupancy'], 'facts': ['fold_change']}
exp4 = {'evidence': ['occupancy'], 'facts': ['activity', 'fold_change']}

experiments = {'exp1': exp1, 'exp4': exp4}

In [11]:
for id, settings in experiments.items():
    train_model.main(
        experiment = EXPERIMENT, 
        path_to_data = f'{PATH_TO_DATA}n1000/mutate/m3/mutate_m3_20/',
        model_template = model, 
        training = 0.8, 
        max_iter = 20, 
        evidence = settings['evidence'],
        facts = settings['facts'],
        suffix = f'_{id}'
    )

Splitting data...
Writing data to files...
Writing evidence to file...
Starting LFI...
problog lfi ./models/mh02/modelA.pl ./models/mh02/evidence_exp1.pl -O ./models/mh02/model_trained_exp1.pl --normalize -k sdd -v -n 20
[INFO] Output level: INFO
[INFO] Number of examples: 800
[INFO] Compile time: 176.253156
[INFO] Weights to learn: [t(0.33,E)::activity(E,S,inc), t(0.33,E)::activity(E,S,dec), t(0.33,E)::activity(E,S,baseline), t(0.33,P)::fold_change(P,S,inc), t(0.33,P)::fold_change(P,S,dec), t(0.33,P)::fold_change(P,S,baseline), t(0.33,E,_dM)::occupancy(P,S,inc), t(0.33,E,_dM)::occupancy(P,S,dec), t(0.33,E,_dM)::occupancy(P,S,baseline), t(0.33,_fc)::occupancy(P,S,inc), t(0.33,_fc)::occupancy(P,S,dec), t(0.33,_fc)::occupancy(P,S,baseline)]
[INFO] Bodies: [lfi_body(0,t(E),E,S,inc), lfi_body(1,t(E),E,S,dec), lfi_body(2,t(E),E,S,baseline), lfi_body(3,t(P),P,S,inc), lfi_body(4,t(P),P,S,dec), lfi_body(5,t(P),P,S,baseline), lfi_body(6,t(E,_dM),P,S,inc), lfi_body(7,t(E,_dM),P,S,dec), lfi_body(

In [6]:
# Testing
test = ['testing']
PATH_TO_TRUTH = f'data/processed/synthetic_data/{MODEL}/n1000/'
results = {}

In [7]:
for exp_id in experiments.keys():
    model_trained = f'{EXP_DIR}model_trained_{exp_id}.pl'
    results[exp_id] = {}
    for test_on in test:
        PATH_TO_TESTING = f'{EXP_DIR}{test_on}_{exp_id}/'
        compare_to = [test_on, 'truth']
        for compare in compare_to:
            if compare == 'truth':
                path_to_truth = PATH_TO_TRUTH
            else: 
                path_to_truth = PATH_TO_TESTING

            test_results = test_model.main(
                experiment = EXPERIMENT, 
                path_to_data = PATH_TO_TESTING, 
                path_to_truth = path_to_truth, 
                model_trained = model_trained, 
                targets = ['occupancy', 'modification'], 
                test_on = f'{test_on}_{exp_id}', compare = f'{compare}_{exp_id}', mode = 'inference')
            results[exp_id][f'{test_on}_vs_{compare}'] = test_results

Inferring data (s10)...
problog ./models/mh02/test_on_testing_exp1.pl -o ./models/mh02/predictions/inferred_example.txt
Finished inference (s10)...
Inferring data (s100)...
problog ./models/mh02/test_on_testing_exp1.pl -o ./models/mh02/predictions/inferred_example.txt
Finished inference (s100)...
Inferring data (s101)...
problog ./models/mh02/test_on_testing_exp1.pl -o ./models/mh02/predictions/inferred_example.txt
Finished inference (s101)...
Inferring data (s109)...
problog ./models/mh02/test_on_testing_exp1.pl -o ./models/mh02/predictions/inferred_example.txt
Finished inference (s109)...
Inferring data (s117)...
problog ./models/mh02/test_on_testing_exp1.pl -o ./models/mh02/predictions/inferred_example.txt
Finished inference (s117)...
Inferring data (s12)...
problog ./models/mh02/test_on_testing_exp1.pl -o ./models/mh02/predictions/inferred_example.txt
Finished inference (s12)...
Inferring data (s126)...
problog ./models/mh02/test_on_testing_exp1.pl -o ./models/mh02/predictions/infe

In [23]:
predictions = {}
for exp_id in experiments.keys():
    PATH_TO_TESTING = f'{EXP_DIR}testing_{exp_id}/'
    model_trained = f'{EXP_DIR}model_trained_{exp_id}.pl'
    predictions[exp_id] = predict_model.main(
        experiment = EXPERIMENT, 
        path_to_data = PATH_TO_TESTING, 
        model_trained = model_trained, 
        targets = ['occupancy', 'modification'],
        mode = 'inference', infer_from = 'facts', 
        suffix = f'_{exp_id}_testing')
results['predictions'] = predictions

Inferring data (s10)...
problog ./models/mh02/make_predictions.pl -o ./models/mh02/predictions/inferred_example.txt
Finished inference (s10)...
Inferring data (s100)...
problog ./models/mh02/make_predictions.pl -o ./models/mh02/predictions/inferred_example.txt
Finished inference (s100)...
Inferring data (s101)...
problog ./models/mh02/make_predictions.pl -o ./models/mh02/predictions/inferred_example.txt
Finished inference (s101)...
Inferring data (s109)...
problog ./models/mh02/make_predictions.pl -o ./models/mh02/predictions/inferred_example.txt
Finished inference (s109)...
Inferring data (s117)...
problog ./models/mh02/make_predictions.pl -o ./models/mh02/predictions/inferred_example.txt
Finished inference (s117)...
Inferring data (s12)...
problog ./models/mh02/make_predictions.pl -o ./models/mh02/predictions/inferred_example.txt
Finished inference (s12)...
Inferring data (s126)...
problog ./models/mh02/make_predictions.pl -o ./models/mh02/predictions/inferred_example.txt
Finished in

In [24]:
with open(f'{EXP_DIR}results.pickle', 'wb') as file:
    pickle.dump(results, file, protocol=pickle.HIGHEST_PROTOCOL) 